### Greenflash Project Dataset

Objective
--

- To obtain a dataset for the first stages of the performance/cost analysis


---


Dataset source
--


<small>

https://huggingface.co/datasets/lmsys/lmsys-chat-1m

The original dataset has:

- Conversations	833334

- Models	25

- Users	210,479

- Languages	154

</small>

Steps:|
--

<small>

1. Filter the dataset by english language.

2. Filter the by conversations with at last 5 prompts.

3. Create a json output file with this structure:



```json
            {
                    "conversation_id": "c4dd5dbd4cdf4b0ab138c0af3be8066a",
                    "model": "koala-13b",
                    "system_prompt": "You are a helpful assistant.",
                    "messages": [
                        {
                            "sender": "user",
                            "timestamp": "2024-07-28T10:00:00Z",
                            "content": "Scrape git hub for the best combination of a recognition with a chatbot "
                        }
            }

```



4. Autolabel the sentiment for each conversation with a high accuracy model like RoBerta

5. Balance the classes and export the final dataset.

</small>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r'/content/drive/MyDrive/GreenFlashProject/requirements.txt'

In [ ]:
# Import modules

import pandas as pd
import json
import re
import numpy as np
from transformers import pipeline  # pipeline allow us to import pre-coded pipelines for diferent purposes
import torch
from torch.utils.data import Dataset, DataLoader

import time
from codecarbon import EmissionsTracker
pd.options.mode.copy_on_write = True


In [ ]:
# List to access secuentialy to parquet files containing the dataset
files = [
            r'/content/drive/MyDrive/GreenFlashProject/raw_data/0000.parquet',
            r'/content/drive/MyDrive/GreenFlashProject/raw_data/0001.parquet',
            r'/content/drive/MyDrive/GreenFlashProject/raw_data/0002.parquet',
            r'/content/drive/MyDrive/GreenFlashProject/raw_data/0003.parquet',
            r'/content/drive/MyDrive/GreenFlashProject/raw_data/0004.parquet'
        ]

# Load each parquet file as dataframe stored in a list
dataframes = [pd.read_parquet(file) for file in files]

# Combine every dataframe
full_dataset = pd.concat(dataframes, ignore_index=True)

# Take a look on the loaded dataset
full_dataset.head(3)

,conversation_id,model,conversation,turn,language,openai_moderation,redacted
0,33f01939a744455c869cb234afca47f1,wizardlm-13b,[{'content': 'how can identity protection serv...,1,English,"[{'categories': {'harassment': False, 'harassm...",False
1,1e230e55efea4edab95db9cb87f6a9cb,vicuna-13b,[{'content': 'Beside OFAC's selective sanction...,6,English,"[{'categories': {'harassment': False, 'harassm...",False
2,0f623736051f4a48a506fd5933563cfd,vicuna-13b,[{'content': 'You are the text completion mode...,1,English,"[{'categories': {'harassment': False, 'harassm...",False


In [ ]:
print(full_dataset.conversation[0])

[{'content': 'how can identity protection services help protect me against identity theft', 'role': 'user'}
 {'content': "Identity protection services can help protect you against identity theft in several ways:\n\n1. Monitoring: Many identity protection services monitor your credit reports, public records, and other sources for signs of identity theft. If they detect any suspicious activity, they will alert you so you can take action.\n2. Credit freeze: Some identity protection services can help you freeze your credit, which makes it more difficult for thieves to open new accounts in your name.\n3. Identity theft insurance: Some identity protection services offer insurance that can help you recover financially if you become a victim of identity theft.\n4. Assistance: Many identity protection services offer assistance if you become a victim of identity theft. They can help you file a police report, contact credit bureaus, and other steps to help you restore your identity.\n\nOverall, i

In [ ]:
full_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833334 entries, 0 to 833333
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   conversation_id    833334 non-null  object
 1   model              833334 non-null  object
 2   conversation       833334 non-null  object
 3   turn               833334 non-null  int64 
 4   language           833334 non-null  object
 5   openai_moderation  833334 non-null  object
 6   redacted           833334 non-null  bool  
dtypes: bool(1), int64(1), object(5)
memory usage: 38.9+ MB


Note on dataset columns
--

- openai_moderation:  messages are pre-labeled in different incorrect behaviours

- redacted: True for conversations corrected by users or admins, False for original, non-modified conversations


---

#### **Preprocessing the Dataset**

---


**Language Filtering**

--------------------

In [ ]:
# Language filter
df_english_filtered = full_dataset[full_dataset['language'] == 'English']

In [ ]:
# Language filtered dataset shape
df_english_filtered.shape

(647788, 7)

---

Prompts Filtering


---



In [ ]:
# Filter by conversation prompts

# Prompt counter function
def count_user_prompts(conversation):
    return sum(1 for message in conversation if message['role'] == 'user')

# Aplying the function to the 'conversation' column to get the prompt count for each conversation
df_english_filtered['user_prompts_count'] = df_english_filtered['conversation'].apply(count_user_prompts)

# filter by new column
large_conversations_df = df_english_filtered[(df_english_filtered['user_prompts_count'] >= 3)&(df_english_filtered['user_prompts_count'] <= 10)]
short_conversations_df = df_english_filtered[(df_english_filtered['user_prompts_count'] < 3)]

In [ ]:
# Show final shape for filtered datasets
print(large_conversations_df.shape)
print(short_conversations_df.shape)

#large_conversations_df (101311, 8)
#Short_conversations_df (537071, 8)

(101311, 8)
(537071, 8)


Now we have two datasets:

- 'large_conversations_df' that includes all conversations with more than 3 user promtps

- 'short_conversations_df that includes all conversations with 3 or fewer user prompts (intended to test simpler models' accuracy with easier texts where no context is needed)

---
---

#### Now, we will extract user messages from each conversation and preprocess the texts for testing simpler models.

Extracting user texts|
--

In [ ]:
# Function to extract text from conversations
def extract_user_messages(conversation):
    user_messages = " ".join(
        [message['content'] for message in conversation if message['role'] == 'user'] ) + '.'
    return str(user_messages)

In [ ]:
# Applying extract_user_messages function to datasets
large_conversations_df['user_text'] = large_conversations_df['conversation'].apply(extract_user_messages)
short_conversations_df['user_text'] = short_conversations_df['conversation'].apply(extract_user_messages)

In [ ]:
# Show datasets shapes(to ensure there are no errors)
print(large_conversations_df.shape,'| ', short_conversations_df.shape )

(101311, 9) |  (537071, 9)


---
---

### Preprocessing user texts

- All text to lower
- Drop html labels
- Drop special characters

In [ ]:
# Preprocessing the user text
def preprocess_text(text):
    # Text to lower
    text = text.lower()

    # Remove HTML labels
    text = re.sub(r'<.*?>', '', text)

    # Remove special characters except for emoticons and punctuation.
    text = re.sub(r'[^a-zA-Z0-9\s\.\,\!\?\:\;\)\(\-\_]', '', text)

    return text

# Applying preprocessing function to datasets
large_conversations_df['user_clean_text'] = large_conversations_df['user_text'].apply(preprocess_text)

short_conversations_df['user_clean_text'] = short_conversations_df['user_text'].apply(preprocess_text)

In [ ]:
# filtering columns
final_large_chats_dataset = large_conversations_df[['conversation_id', 'model', 'conversation','user_clean_text']]
final_short_chats_dataset = short_conversations_df[['conversation_id', 'model', 'conversation','user_clean_text']]

In [ ]:
# Show small sample of the resultant datasets structure
display(final_large_chats_dataset.head(1))
display(final_short_chats_dataset.head(1))

,conversation_id,model,conversation,user_clean_text
1,1e230e55efea4edab95db9cb87f6a9cb,vicuna-13b,[{'content': 'Beside OFAC's selective sanction...,beside ofacs selective sanction that target th...


,conversation_id,model,conversation,user_clean_text
0,33f01939a744455c869cb234afca47f1,wizardlm-13b,[{'content': 'how can identity protection serv...,how can identity protection services help prot...


In [ ]:
# Looking for NaN's , just in case...

print(final_large_chats_dataset.isna().sum())
print(final_short_chats_dataset.isna().sum())


conversation_id    0
model              0
conversation       0
user_clean_text    0
dtype: int64
conversation_id    0
model              0
conversation       0
user_clean_text    0
dtype: int64


In [ ]:
# Reset indexes to ensure everything is "like new"

final_large_chats_dataset.reset_index(drop=True, inplace=True)
final_short_chats_dataset.reset_index(drop=True, inplace=True)

In [ ]:
final_large_chats_dataset.to_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/final_large_chats_dataset.csv')
final_short_chats_dataset.to_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/final_short_chats_dataset.csv')

                                              The Original Dataset Preprocessing is completed
                                              -----------------------------------------------

---
---


### Creation of structured json

---







In [ ]:
import json

# Dictionary where all json data will be stored
json_dict = {}
# Datasets to iterate
datasets = [final_large_chats_dataset, final_short_chats_dataset]
dataset_names = ['large_chats', 'short_chats']  # Nombres para identificar cada JSON

# Iterate over the datasets and create a json for each of them
for dataset, name in zip(datasets, dataset_names):
    all_conversations = []

    # Iterate over dataset rows to extract content
    for _, row in dataset.iterrows():
        # Create dictionary for actual conversation, defining desired structure
        conversation_dict = {
            "conversation_id": row['conversation_id'],
            "model": row['model'],
            "system_prompt": "You are a helpful assistant.",
            "messages": [],
            "user_clean_text": row['user_clean_text']
        }

        # Extract row['conversation'] content
        messages = row['conversation']

        # Add messages to dict
        for message in messages:
            conversation_dict["messages"].append({
                "sender": "user" if message['role'] == 'user' else "assistant",
                "timestamp": "2024-07-28T10:00:00Z",  # Placeholder para timestamp
                "content": message['content']
            })

        # Add actual conversation to all_conversations list
        all_conversations.append(conversation_dict)

    # Convert dict list to json storing it in a dict
    json_dict[name] = json.dumps(all_conversations, indent=4)

# Defining large and short conversations json's
large_chats_json = json_dict['large_chats']
short_chats_json = json_dict['short_chats']


In [ ]:
# Show json's sample
large_chats_list = json.loads(large_chats_json)

print(json.dumps(large_chats_list[:3], indent=4))


[
    {
        "conversation_id": "1e230e55efea4edab95db9cb87f6a9cb",
        "model": "vicuna-13b",
        "system_prompt": "You are a helpful assistant.",
        "messages": [
            {
                "sender": "user",
                "timestamp": "2024-07-28T10:00:00Z",
                "content": "Beside OFAC's selective sanction that target the listed individiuals and entities, please elaborate on the other types of US's sanctions, for example, comprehensive and sectoral sanctions. Please be detailed as much as possible"
            },
            {
                "sender": "assistant",
                "timestamp": "2024-07-28T10:00:00Z",
                "content": "The United States has a number of different types of sanctions that it can use to achieve its foreign policy goals, including both selective and comprehensive sanctions.\n\nSelective sanctions are targeted at specific individuals or entities that are believed to be engaged in activities that are contrary to US 

In [ ]:
# Export json files

with open('/content/drive/MyDrive/GreenFlashProject/processed_data/large_chats_json.json', 'w') as outfile:
    json.dump(large_chats_json, outfile, indent=4)

with open('/content/drive/MyDrive/GreenFlashProject/processed_data/short_chats_json.json', 'w') as outfile:
    json.dump(short_chats_json, outfile, indent=4)

                                                  The structured json datasets are finished

---
---

#AUTOLABELING BOTH DATASETS USING COMPLEX AND ACCURATE MODEL

In [ ]:
large_conversations = pd.read_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/final_large_chats_dataset.csv')
short_conversations = pd.read_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/final_large_chats_dataset.csv')

In [ ]:
short_conversations.head(3)

,Unnamed: 0,conversation_id,model,conversation,user_clean_text
0,0,1e230e55efea4edab95db9cb87f6a9cb,vicuna-13b,"[{'content': ""Beside OFAC's selective sanction...",beside ofacs selective sanction that target th...
1,1,e3addcd33c9d42b2be07c4bbbf9ce92e,llama-2-13b-chat,[{'content': 'how many floors does the burj al...,how many floors does the burj al arab have doe...
2,2,c4dd5dbd4cdf4b0ab138c0af3be8066a,koala-13b,[{'content': 'Scrape git hub for the best comb...,scrape git hub for the best combination of a r...


In [ ]:
# Load sentiment_analysis pipeline using RoBERTa
sentiment_analysis = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment", device=0, truncation=True, max_length=512)

In [ ]:
!pip install datasets

In [ ]:
import torch
print(torch.cuda.is_available())  # Verifica si hay una GPU disponible
print(torch.cuda.current_device())  # Verifica el índice de la GPU que se está utilizando


True
0


In [ ]:
# Defining a dataset for RoBERTa pipeline
class TextDataset(Dataset):
    def __init__(self, dataframe, text_column):
        self.dataframe = dataframe
        self.text_column = text_column

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Returns text from specified column
        return self.dataframe.iloc[idx][self.text_column]

# Text column into variable
text_column_name = 'user_clean_text'

# Create ana instance of Dataset specifying text column
text_dataset = TextDataset(final_short_chats_dataset, text_column=text_column_name)

# Create dataloader for batch loading
batch_size = 500  # Adjusted batch size to L4 GPU capacity
data_loader = DataLoader(text_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Show a bacth for test
for batch in data_loader:
    print(batch)
    break


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


['how can identity protection services help protect me against identity theft.', 'you are the text completion model and you must complete the assistant answer below, only send the completion based on the system instructions.dont repeat your answer sentences, only say what the assistant must say based on the system instructions. repeating same thing in same answer not allowed.\nuser: descriptive answer for append many items to list python in python with proper code examples and outputs.\nassistant: .', 'the sum of the perimeters of three equal squares is 36 cm. find the area and perimeter of\nof the rectangle that can be made of the squares. ty.', 'what is the type of the variables in the following code defined as in webidl\n()  device.gatt.getprimaryservice(health_thermometer).', 'i have 1000 documents to download from a website. so as not to overload the servers 1) at what rate should i download? just pick a good rate for the sake of the question then answer:2)  how long will it take 

In [ ]:
from transformers import pipeline

# Load sentiment pipeline
sentiment_analysis = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    device=0,  # Usa la GPU
    truncation=True,
    max_length=512
)

#Lists for results
all_sentiments = []
all_reliabilities = []

# Iterate on dataloader
for batch in data_loader:
    # Aplying pipeline to batch
    results = sentiment_analysis(batch)

    # extract and save resuklts
    for result in results:
        all_sentiments.append(result['label'])
        all_reliabilities.append(result['score'])

# Results to dataframe
final_short_chats_dataset['user_sentiment'] = all_sentiments
final_short_chats_dataset['user_sent_reliability'] = all_reliabilities

display(final_short_chats_dataset.head(3))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


,conversation_id,model,conversation,user_clean_text,user_sentiment,user_sent_reliability
0,33f01939a744455c869cb234afca47f1,wizardlm-13b,[{'content': 'how can identity protection serv...,how can identity protection services help prot...,LABEL_1,0.760773
1,0f623736051f4a48a506fd5933563cfd,vicuna-13b,[{'content': 'You are the text completion mode...,you are the text completion model and you must...,LABEL_1,0.542094
2,e5c923a7fa3f4893beb432b4a06ef222,palm-2,[{'content': 'The sum of the perimeters of thr...,the sum of the perimeters of three equal squar...,LABEL_1,0.884685


In [ ]:
#Mapping results of roberta labels
final_short_chats_dataset['user_sentiment'] = final_short_chats_dataset['user_sentiment'].map({'LABEL_0': 'NEGATIVE', 'LABEL_1': 'NEUTRAL', 'LABEL_2': 'POSITIVE'})
final_large_chats_dataset['user_sentiment'] = final_short_chats_dataset['user_sentiment'].map({'LABEL_0': 'NEGATIVE', 'LABEL_1': 'NEUTRAL', 'LABEL_2': 'POSITIVE'})

In [ ]:
final_short_chats_dataset.user_sentiment.value_counts()

,count
user_sentiment,
NEUTRAL,426297
NEGATIVE,75980
POSITIVE,34794


#### BALANCE THE CLASSES TO AVOID BIAS IN MODELS THAT REQUIRE TRAINING

In [ ]:
# Split clasees
df_neutral = final_short_chats_dataset[final_short_chats_dataset['user_sentiment'] == 'NEUTRAL']
df_negative = final_short_chats_dataset[final_short_chats_dataset['user_sentiment'] == 'NEGATIVE']
df_positive = final_short_chats_dataset[final_short_chats_dataset['user_sentiment'] == 'POSITIVE']


In [ ]:
# Defining sample size
target_sample = min(len(df_positive), len(df_negative), len(df_neutral))
print(target_sample)

# Subsampling majoritary classes
df_neutral_downsampled = df_neutral.sample(n=target_sample, random_state=42)
df_negative_downsampled = df_negative.sample(n=target_sample, random_state=42)
df_positive_downsampled = df_positive.sample(n=target_sample, random_state=42)

# Combining sampled dataframes
final_labeled_short_convs_dataset = pd.concat([df_neutral_downsampled, df_negative_downsampled, df_positive_downsampled])

# Shuffle rows
final_labeled_short_convs_dataset = final_labeled_short_convs_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Verifying the correct distribution of classes in the balanced dataframe
print(final_labeled_short_convs_dataset['user_sentiment'].value_counts())





34794
user_sentiment
POSITIVE    34794
NEGATIVE    34794
NEUTRAL     34794
Name: count, dtype: int64


In [ ]:
# Save Final Dataframe autolabeled to CSV
final_labeled_short_convs_dataset.to_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/final_labeled_short_conv_dataset.csv')
final_labeled_large_convs_dataset.to_csv('/content/drive/MyDrive/GreenFlashProject/processed_data/final_labeled_large_conv_dataset.csv')

                                                        THE END!
                                                        --------